<a href="https://colab.research.google.com/github/Kim-Yeon-Jun/Dacon/blob/main/Daegu_231128_442.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
import numpy as np
import pandas as pd
from torch import nn
from torch import optim
from torch.utils.data import DataLoader, Dataset
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from torch.utils.data import TensorDataset, DataLoader

In [2]:
device = "cuda" if torch.cuda.is_available() else "cpu"

In [3]:
df = pd.read_csv("/content/drive/MyDrive/Dacon/Daegu_accident/df_train.csv")

In [4]:
df2 = pd.read_csv("/content/drive/MyDrive/Dacon/Daegu_accident/df_test.csv")

In [5]:
df.shape

(39601, 41)

In [6]:
df2.shape

(10963, 37)

In [7]:
df.drop(['구'], axis=1, inplace = True)

In [8]:
df2.drop(['구'], axis = 1, inplace = True)

In [ ]:
df.iloc[1:20,1:5]

,사망자수,중상자수,경상자수,부상자수
1,0,0,1,0
2,0,0,1,0
3,0,1,0,0
4,0,0,1,0
5,0,0,2,0
6,0,0,6,0
7,0,0,1,0
8,0,1,0,0
9,0,0,3,0
10,0,1,0,0


In [ ]:
df2.head()

,Unnamed: 0,주말,평일,기타,눈,맑음,비,흐림,남구,달서구,...,주차장 - 주차장,건조,기타.1,서리/결빙,적설,젖음/습기,침수,차대사람,차대차,차량단독
0,0,1,0,0,0,1,0,0,0,0,...,0,1,0,0,0,0,0,1,0,0
1,1,1,0,0,0,1,0,0,0,0,...,0,1,0,0,0,0,0,1,0,0
2,2,1,0,0,0,1,0,0,0,0,...,0,1,0,0,0,0,0,0,1,0
3,3,1,0,0,0,1,0,0,0,0,...,0,1,0,0,0,0,0,0,1,0
4,4,1,0,0,0,1,0,0,0,1,...,0,1,0,0,0,0,0,0,1,0


In [9]:
df.drop(['Unnamed: 0'], inplace=True, axis = 1)
df2.drop(['Unnamed: 0'], inplace=True, axis = 1)

In [10]:
# X_test = df2.copy()
X = df.drop(['사망자수','중상자수','경상자수','부상자수'], axis=1).copy()
y = df[['사망자수','중상자수','경상자수','부상자수']].copy()

In [11]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state =42)

In [12]:
X_train = X_train.to_numpy()
X_test = X_test.to_numpy()
y_train = y_train.to_numpy()
y_test = y_test.to_numpy()

In [13]:
X_train = torch.tensor(X_train, dtype=torch.float32)
X_test = torch.tensor(X_test, dtype=torch.float32)
y_train = torch.tensor(y_train, dtype=torch.float32)
y_test = torch.tensor(y_test, dtype=torch.float32)

In [14]:
train_dataset = TensorDataset(X_train, y_train)
test_dataset = TensorDataset(X_test, y_test)

In [15]:
train_dataloader = DataLoader(train_dataset, batch_size = 64, shuffle = True)
test_dataloader = DataLoader(test_dataset, batch_size= 16, shuffle = True)

In [16]:
class CustomModel(nn.Module):
  def __init__(self, input_size, output_size):
    super(CustomModel, self).__init__()
    self.fc1 = nn.Linear(input_size, 64)
    self.fc2 = nn.BatchNorm1d(64)
    self.fc3 = nn.Dropout(p=0.3)
    self.fc4 = nn.Linear(64, 32)
    self.fc5 = nn.BatchNorm1d(32)
    self.fc6 = nn.Linear(32, output_size)

  def forward(self, x):
    x = torch.relu(self.fc1(x))
    x = torch.relu(self.fc2(x))
    x = torch.relu(self.fc3(x))
    x = torch.relu(self.fc4(x))
    x = torch.relu(self.fc5(x))
    x = self.fc6(x)
    return x

In [17]:
device = "cuda" if torch.cuda.is_available() else "cpu"
input_size = 35
output_size = 4
model = CustomModel(input_size, output_size).to(device)
criterion = nn.MSELoss().to(device)
optimizer = optim.Adam(model.parameters(), lr = 0.01)

In [18]:
num_epochs = 10

for epoch in range(num_epochs):
    model.train()
    total_loss = 0.0

    for x, y in train_dataloader:
        x = x.to(device)
        y = y.to(device)

        optimizer.zero_grad()
        outputs = model(x)

        loss = criterion(outputs, y)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    # 일정 간격마다 예측값 출력
    if (epoch + 1) % 5 == 0:
        model.eval()
        with torch.no_grad():
            inputs = torch.randn(5, input_size).to(device)  # 예측값을 확인할 임의의 입력 데이터
            predictions = torch.sigmoid(model(inputs))
            print(f"Epoch [{epoch + 1}/{num_epochs}], Loss: {total_loss / len(train_dataloader):.4f}")
            print("Predictions:")
            print(predictions)



Epoch [5/10], Loss: 0.3316
Predictions:
tensor([[0.8144, 0.3456, 0.9732, 0.9550],
        [0.7531, 0.2739, 0.9943, 0.9834],
        [0.6878, 0.9015, 0.9998, 0.9470],
        [0.5429, 0.5867, 0.8667, 0.6145],
        [0.9852, 0.6368, 0.9984, 0.9964]])
Epoch [10/10], Loss: 0.3315
Predictions:
tensor([[0.5131, 0.6200, 0.6286, 0.6030],
        [0.9455, 0.9979, 0.2922, 1.0000],
        [0.5169, 0.6206, 0.6252, 0.6082],
        [0.7068, 0.8838, 0.4031, 0.8347],
        [0.6307, 0.7162, 0.9846, 0.6284]])


In [20]:
print(model.state_dict)

<bound method Module.state_dict of CustomModel(
  (fc1): Linear(in_features=35, out_features=64, bias=True)
  (fc2): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (fc3): Dropout(p=0.3, inplace=False)
  (fc4): Linear(in_features=64, out_features=32, bias=True)
  (fc5): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (fc6): Linear(in_features=32, out_features=4, bias=True)
)>


In [24]:
sub = df2.to_numpy()
sub = torch.tensor(sub, dtype=torch.float32)
sub_dataset = TensorDataset(sub)
sub_dataloader = DataLoader(sub_dataset, batch_size=1, shuffle=False)

In [29]:
# 최종 학습 결과 확인
model.eval()
predictions_list = []
with torch.no_grad():
  for inputs in sub_dataloader:
    #inputs.to(deivce)
    predictions = model(inputs[0])
    rpredictions = torch.round(predictions)
    predictions_list.append(rpredictions.numpy())
    # print("Final Predictions:")
    # print(rpredictions)

predictions_array = np.concatenate(predictions_list, axis = 0)
columns = ['사망', '중상', '경상','부상']
predictions_df = pd.DataFrame(predictions_array, columns=columns)
predictions_df.to_csv('/content/drive/MyDrive/Dacon/Daegu_accident/predictions.csv', index=False)

In [ ]:
# class NeuralNetwork(nn.Module):
#   def __init__(self):
#     super(NeuralNetwork, self).__init__()
#     self.input_layer = nn.Linear(4, 16)
#     self.hidden_layer1 = nn.Linear(16,32)
#     self.output_layer = nn.Linear(32, 3)
#     self.relu = nn.ReLU()

#   def forward(self,x):
#     out = self.relu(self.input_layer(x))
#     out = self.relu(self.hidden_layer1(out))
#     out = self.output_layer(out)
#     return out

In [30]:
df3 = pd.read_csv('/content/drive/MyDrive/Dacon/Daegu_accident/predictions.csv')

In [31]:
df3.head()

,사망,중상,경상,부상
0,0.0,0.0,1.0,0.0
1,0.0,0.0,1.0,0.0
2,0.0,0.0,1.0,0.0
3,0.0,0.0,1.0,0.0
4,0.0,0.0,1.0,0.0


In [33]:
df3['ECLO'] = df3['사망'] * 10 + df3['중상'] * 5 + df3['경상'] * 3 + df3['부상'] * 1

In [34]:
df3.head()

,사망,중상,경상,부상,ECLO
0,0.0,0.0,1.0,0.0,3.0
1,0.0,0.0,1.0,0.0,3.0
2,0.0,0.0,1.0,0.0,3.0
3,0.0,0.0,1.0,0.0,3.0
4,0.0,0.0,1.0,0.0,3.0


In [35]:
df3.ECLO.value_counts()

3.0    10940
6.0       20
0.0        2
4.0        1
Name: ECLO, dtype: int64

In [36]:
df4 = pd.read_csv('/content/drive/MyDrive/Dacon/Daegu_accident/sample_submission.csv')

In [37]:
df4['ECLO'] = df3.ECLO.copy()

In [38]:
df4.head()

,ID,ECLO
0,ACCIDENT_39609,3.0
1,ACCIDENT_39610,3.0
2,ACCIDENT_39611,3.0
3,ACCIDENT_39612,3.0
4,ACCIDENT_39613,3.0


In [39]:
df4.to_csv('/content/drive/MyDrive/Dacon/Daegu_accident/sample_submission1.csv', index=False)

In [40]:
df4.tail()

,ID,ECLO
10958,ACCIDENT_50567,6.0
10959,ACCIDENT_50568,3.0
10960,ACCIDENT_50569,3.0
10961,ACCIDENT_50570,3.0
10962,ACCIDENT_50571,3.0
